In [8]:
# !pip install torch
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)


In [27]:
import os

data_dir = "imagens"
print(os.listdir(data_dir))
num_classes = len(os.listdir(data_dir))
print("Número de classes:", num_classes)

['Butterfly', 'Dragonfly', '.ipynb_checkpoints']
Número de classes: 3


In [36]:
# Arquitetura do modelo

import torch
import torch.nn as nn

class InsectClassifier(nn.Module):
    def __init__(self, num_classes):
        super(InsectClassifier, self).__init__()
        
        # Camadas convolucionais
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Camadas totalmente conectadas
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 56 * 56, 128),  # Correção no tamanho do tensor de entrada
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Achatando o tensor para a camada totalmente conectada
        x = self.fc_layers(x)
        return x

# Criando uma instância do modelo
num_classes = num_classes
model = InsectClassifier(num_classes)

# Imprimindo a arquitetura do modelo
print(model)

InsectClassifier(
  (conv_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=100352, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=3, bias=True)
  )
)


In [38]:
# Preparar os dados
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Caminho para a pasta que contém as subpastas das classes
data_dir = "imagens"

# Define as transformações para pré-processamento das imagens
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Carrega os dados usando ImageFolder
dataset = ImageFolder(data_dir, transform=data_transforms)

# Divide os dados em conjuntos de treinamento, validação e teste
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Cria DataLoaders para os conjuntos de treinamento e validação
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
print(train_loader)

In [39]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [40]:
import torch.optim as optim

# Defina a função de perda (loss function) e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()  # Define o modelo para modo de treinamento
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calcula a perda média durante a época
    epoch_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

    # Avalia o modelo no conjunto de validação
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Calcula a perda e a precisão médias no conjunto de validação
    val_loss /= len(val_loader)
    val_accuracy = correct / total

    print(f"Validation Loss: {val_loss:.4f} - Accuracy: {val_accuracy:.4f}")


Epoch [1/10] - Loss: 4.8522
Validation Loss: 4.4425 - Accuracy: 0.5484
Epoch [2/10] - Loss: 2.3594
Validation Loss: 2.8997 - Accuracy: 0.4516
Epoch [3/10] - Loss: 1.6301
Validation Loss: 0.5163 - Accuracy: 0.7097
Epoch [4/10] - Loss: 0.3346
Validation Loss: 0.4548 - Accuracy: 0.8387
Epoch [5/10] - Loss: 0.3720
Validation Loss: 0.3070 - Accuracy: 0.8387
Epoch [6/10] - Loss: 0.2519
Validation Loss: 0.3582 - Accuracy: 0.8710
Epoch [7/10] - Loss: 0.1665
Validation Loss: 0.5153 - Accuracy: 0.7097
Epoch [8/10] - Loss: 0.1360
Validation Loss: 0.4755 - Accuracy: 0.7742
Epoch [9/10] - Loss: 0.0929
Validation Loss: 0.4718 - Accuracy: 0.7419
Epoch [10/10] - Loss: 0.0673
Validation Loss: 0.4390 - Accuracy: 0.8065


In [42]:
# Caminho onde você deseja salvar o modelo treinado
save_path = "modelo_treinado.pth"

# Salvar o estado do modelo
torch.save(model.state_dict(), save_path)

print(f"Modelo treinado salvo em {save_path}")

Modelo treinado salvo em modelo_treinado.pth


In [44]:
model = InsectClassifier(num_classes)
model.load_state_dict(torch.load(save_path))
model.to(device)

# Preparar os dados de teste
test_dataset = ImageFolder("imagens_teste", transform=data_transforms)

# Criar DataLoader para o conjunto de teste
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Carregar o modelo treinado
model.load_state_dict(torch.load("modelo_treinado.pth"))
model.to(device)

# Avaliar o modelo no conjunto de teste
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")


Test Accuracy: 0.6399


In [50]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Carregar o modelo treinado
model = InsectClassifier(num_classes)
model.load_state_dict(torch.load("modelo_treinado.pth"))
model.to(device)
model.eval()

# Carregar e pré-processar a imagem de entrada
image_path = "google141.jpg"
image = Image.open(image_path).convert("RGB")
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = data_transform(image).unsqueeze(0).to(device)

# Fazer a previsão
with torch.no_grad():
    outputs = model(input_image)
    predicted_class = torch.argmax(outputs, dim=1).item()

# Mapear o índice da classe para o nome da classe
class_names = dataset.classes
predicted_class_name = class_names[predicted_class]

print(f"A imagem pertence à classe: {predicted_class_name}")


A imagem pertence à classe: Butterfly
